Импортируем нужные библиотеки.

In [1]:
import pickle
import numpy as np
from sympy import *
import math
from math import copysign as sgn, factorial as fact
from mpmath import besselj as Jn
from sympy.physics.quantum.cg import CG
from scipy.special import gamma, binom, sph_harm as Ynm, gammaincc
from scipy.special import spherical_jn
from scipy import integrate
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Считаем из файлов координаты точек сетки интегрирования, значения электронной плотности в них и координаты атомов в молекуле.

In [2]:
with open("grid.bin", "rb") as file:
    coords = pickle.load(file)

with open("NH2_coords.bin", "rb") as file:
    geometry = pickle.load(file)
    
with open("density.bin", "rb") as file:
    density = pickle.load(file)

In [3]:
N_R=150
N_Theta=35
N_Phi=72

X=np.reshape(coords[:,0]*np.sin(coords[:,1])*np.cos(coords[:,2]),(len(coords),1))
Y=np.reshape(coords[:,0]*np.sin(coords[:,1])*np.sin(coords[:,2]),(len(coords),1))
Z=np.reshape(coords[:,0]*np.cos(coords[:,1]),(len(coords),1))
coords_cartesian = np.concatenate([X,Y,Z],axis=1)

R_step=coords[N_Theta*N_Phi,0]-coords[0,0]
Theta_step=coords[N_Phi,1]-coords[0,1]
Phi_step=coords[1,2]-coords[0,2]

Рассчитаем значения потенциала ядер-электронного взаимодействия в узлах сетки интегрирования.

In [4]:
R = np.sqrt(np.sum((coords_cartesian[:,None,:]-geometry[None,:,1:])**2,axis=-1))
Ven = np.sum(-1*geometry[:,0]/R[:,:],axis=-1)
Ven.shape

(378000,)

Зададим массив весов для интегрирования методом трапеций.

In [5]:
weights=[]
h1=h2=h3=1
for i in range (N_R):
    if ((i==0) or (i==N_R-1)):
        h1=0.5
    else:
        h1=1
    for j in range (N_Theta):
        if ((j==0) or (j==N_Theta-1)):
            h2=0.5
        else:
            h2=1
        for k in range (N_Phi):
            if ((k==0) or (k==N_Phi-1)):
                h3=0.5
            else:
                h3=1
            weights.append(h1*h2*h3)
weights = np.array(weights)
weights.shape

(378000,)

Рассчитаем значения потенциала электрон-электронного взаимодействия в узлах сетки интегрирования. 

In [6]:
from joblib import Parallel, delayed
weight_dens = R_step*Theta_step*Phi_step*density*weights*coords[:,0]**2*np.sin(coords[:,1])
NCPUS = 4 #Количество ядер, на которых будет проводится расчет
result = Parallel(n_jobs=NCPUS, verbose=5)(delayed( lambda i : np.sum(np.delete(weight_dens, i, axis=0)\
                /np.sqrt(np.sum((coords_cartesian[i,:]-np.delete(coords_cartesian, i, axis=0))**2,axis=1))))(i)\
                for i in range(len(coords)))

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 156 tasks      | elapsed:    2.4s
[Parallel(n_jobs=4)]: Done 516 tasks      | elapsed:    6.1s
[Parallel(n_jobs=4)]: Done 1020 tasks      | elapsed:   11.2s
[Parallel(n_jobs=4)]: Done 1668 tasks      | elapsed:   17.5s
[Parallel(n_jobs=4)]: Done 2460 tasks      | elapsed:   25.4s
[Parallel(n_jobs=4)]: Done 3396 tasks      | elapsed:   35.8s
[Parallel(n_jobs=4)]: Done 4476 tasks      | elapsed:   46.4s
[Parallel(n_jobs=4)]: Done 5700 tasks      | elapsed:   58.9s
[Parallel(n_jobs=4)]: Done 7068 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done 8580 tasks      | elapsed:  1.5min
[Parallel(n_jobs=4)]: Done 10236 tasks      | elapsed:  1.8min
[Parallel(n_jobs=4)]: Done 12036 tasks      | elapsed:  2.1min
[Parallel(n_jobs=4)]: Done 13980 tasks      | elapsed:  2.4min
[Parallel(n_jobs=4)]: Done 16068 tasks      | elapsed:  2

In [7]:
Vee=np.array(result)
Vee.shape

(378000,)

Считаем из бинарного файла значения обменно-корреляционного потенциала.

In [11]:
with open("Vxc.bin", "rb") as file:
    Vxc = pickle.load(file)
Vxc

array([-6.15726196e-01, -6.15927107e-01, -6.16126570e-01, ...,
       -5.32441111e-09, -5.29962220e-09, -5.27162073e-09])

Зададим суммарный потенциал: $V_{\text{tot}}=V_{\text{ee}}+V_{\text{eN}}+V_{\text{xc}}$.

In [12]:
Vtot = np.sum([Ven,Vee,Vxc],axis=0)
Vtot.shape

(378000,)

Сохраним в бинарный файл значения суммарного потенциала на сетке интегрирования.

In [14]:
with open("Vtot.bin", "wb") as file:
    pickle.dump(Vtot, file)